In [1]:
import itertools as it

from matplotlib import pyplot as plt
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp


In [2]:
# prints metadata about notebook runtime
nbm.print_metadata()


context: ci
hostname: fv-az569-92
interpreter: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
nbcellexec: null
nbname: postprocess-bias
nbpath: /home/runner/work/hstrat-evolutionary-inference/hstrat-evolutionary-inference/binder/simple-splayed/postprocess-bias.ipynb
revision: null
session: a8c1e8d4-4e77-4d1b-97e6-10614245eef5
timestamp: 2024-03-26T11:37:49Z00:00


IPython==8.13.0
keyname==0.5.2
yaml==6.0
matplotlib==3.7.1
nbmetalog==0.2.6
numpy==1.21.6
pandas==1.3.5
seaborn==0.12.1
teeplot==1.0.1
re==2.2.1
comm==0.2.2
logging==0.5.1.2
psutil==5.9.8
zmq==24.0.1
json==2.0.9
ipykernel==6.29.3


In [3]:
df = pd.read_csv("https://osf.io/xd95a/download")
dfdigest = np.bitwise_xor.reduce(
    pd.util.hash_pandas_object(df),
)
print("{:x}".format(dfdigest))
df


9554d22d80ec66e7


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,variance_pairwise_distance,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess
0,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,36,...,46.889809,46.889809,0,40510,1210.960392,3.210256e+06,504052.827821,NaN,NaN,NaN
1,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,37,...,49.673715,49.673715,0,60605,1056.875368,3.230868e+06,865335.516754,NaN,NaN,NaN
2,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,6,...,34.243441,34.243441,0,39137,1151.469136,3.171146e+06,727694.363524,NaN,NaN,NaN
3,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,20,...,37.225022,37.225022,0,40397,1289.021227,3.339854e+06,436404.480172,NaN,NaN,NaN
4,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,34,...,35.771091,35.771091,0,44263,1217.977803,3.237385e+06,540209.820753,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108970,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,28.560595,28.560595,1,3547,99.163609,3.242650e+04,282.320163,30.0,0.01,naive
108971,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,16,...,20.627240,20.627240,0,3331,1893.146789,6.190590e+05,250803.885040,100.0,0.01,rollback
108972,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,1,...,114.169456,114.169456,1,821866,5.744125,1.882235e+05,6.163179,100.0,1.00,rollback
108973,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,36.528638,36.528638,0,623208,28.659882,9.391270e+05,116.954707,3.0,1.00,naive


In [4]:
# filter out incomplete sets of replicates
df = df.groupby(
    [
        'epoch',
        'mut_distn',
        'num_islands',
        'num_niches',
        'p_island_migration',
        'p_niche_invasion',
        'population_size',
        'tournament_size',
        'treatment',
        # 'resolution',
        # 'subsampling-fraction',
        # 'trie-postprocess',
    ],
).filter(
    lambda x: len(x) >= 50,
).reset_index(drop=True)
df


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,variance_pairwise_distance,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess
0,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,36,...,46.889809,46.889809,0,40510,1210.960392,3.210256e+06,504052.827821,NaN,NaN,NaN
1,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,37,...,49.673715,49.673715,0,60605,1056.875368,3.230868e+06,865335.516754,NaN,NaN,NaN
2,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,6,...,34.243441,34.243441,0,39137,1151.469136,3.171146e+06,727694.363524,NaN,NaN,NaN
3,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,20,...,37.225022,37.225022,0,40397,1289.021227,3.339854e+06,436404.480172,NaN,NaN,NaN
4,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,34,...,35.771091,35.771091,0,44263,1217.977803,3.237385e+06,540209.820753,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108970,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,28.560595,28.560595,1,3547,99.163609,3.242650e+04,282.320163,30.0,0.01,naive
108971,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,16,...,20.627240,20.627240,0,3331,1893.146789,6.190590e+05,250803.885040,100.0,0.01,rollback
108972,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,1,...,114.169456,114.169456,1,821866,5.744125,1.882235e+05,6.163179,100.0,1.00,rollback
108973,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,36.528638,36.528638,0,623208,28.659882,9.391270e+05,116.954707,3.0,1.00,naive


In [5]:
# phylometrics whittled down to descriptive, non-redundant set
df["colless-like index"] = df["colless_like_index"]
df["mean pairwise distance"] = df["mean_pairwise_distance"]
df["sum distance"] = df["sum_distance"]
df["mean evolutionary distinctiveness"] = df[
    "mean_evolutionary_distinctiveness"
]

phylometrics = [
    #     'diversity',
    "colless-like index",
    #     'average_depth',
    #     'average_origin_time',
    #     'average_origin_time_normalized',
    #     'max_depth',
    "mean pairwise distance",
    #     'mean_pairwise_distance_branch_only',
    #     'num_active',
    "sum distance",
    # "num_ancestors",
    #     'num_outside',
    #     'num_roots',
    #     'num_taxa',
    #     'phylogenetic_diversity',
    #     'sum_pairwise_distance',
    #     'sum_pairwise_distance_branch_only',
    #     'total_orgs',
    #     'tree_size',
    #     'variance_pairwise_distance',
    #     'variance_pairwise_distance_branch_only',
    #     'mrca_depth',
    #     'sackin_index',
    "mean evolutionary distinctiveness",
    #     'sum_evolutionary_distinctiveness',
    #     'variance_evolutionary_distinctiveness',
]


In [6]:
methodological_variables = [
    "resolution",
    "trie-postprocess",
]


In [7]:
evolutionary_variables = [
    "num_islands",
    "num_niches",
    "tournament_size",
]


In [8]:
sensitivity_analysis_variables = [
    "epoch",
    "mut_distn",
]


In [9]:
fixed_variables = [
    "num_generations",  # homogeneous (num generations per epoch)
    "p_island_migration",  # homogeneous
    "population_size",  # homogeneous
    "p_niche_invasion",  # only use 3.0517578125e-08
    "subsampling-fraction",  # only use 1.0
]


In [10]:
# ensure fixed variables
df = df[
    (df["p_island_migration"] == 0.01)
    & (df["p_niche_invasion"] == 3.0517578125e-08)
    & ((df["subsampling-fraction"] == 1.0) | df["subsampling-fraction"].isna())
].copy()

for fixed_variable in fixed_variables:
    assert len(df[fixed_variable].dropna().unique()) == 1, excluded_variable

df


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess,colless-like index,mean pairwise distance,sum distance,mean evolutionary distinctiveness
0,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,36,...,1210.960392,3.210256e+06,504052.827821,NaN,NaN,NaN,25175.104992,26.680483,3.120820e+06,1210.960392
1,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,37,...,1056.875368,3.230868e+06,865335.516754,NaN,NaN,NaN,48090.905691,27.383139,3.151556e+06,1056.875368
2,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,6,...,1151.469136,3.171146e+06,727694.363524,NaN,NaN,NaN,20494.479665,25.474695,3.086073e+06,1151.469136
3,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,20,...,1289.021227,3.339854e+06,436404.480172,NaN,NaN,NaN,26569.469529,25.161812,3.249705e+06,1289.021227
4,consolidated-phylogeny,2,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,34,...,1217.977803,3.237385e+06,540209.820753,NaN,NaN,NaN,30006.265558,25.553675,3.152357e+06,1217.977803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108968,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,18,...,5.482509,1.796509e+05,4.899477,10.0,1.0,expected,514234.752037,42.188129,2.421064e+05,5.482509
108969,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,23,...,3.439714,1.127125e+05,1.327019,100.0,1.0,expected,480536.545977,41.468061,1.963595e+05,3.439714
108972,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,1,...,5.744125,1.882235e+05,6.163179,100.0,1.0,rollback,616421.388552,45.339285,2.449270e+05,5.744125
108973,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,28.659882,9.391270e+05,116.954707,3.0,1.0,naive,335953.828373,34.752739,9.391270e+05,28.659882


In [11]:
# exclude extraneous variable values
df = df[(df["a"] != "consolidated-phylogeny")].copy()

df


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess,colless-like index,mean pairwise distance,sum distance,mean evolutionary distinctiveness
300,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,46,...,118.941956,3.897490e+06,4181.014677,10.0,1.0,rollback,305700.072677,32.806890,3.897490e+06,118.941956
301,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,30,...,123.441128,4.044919e+06,5416.806830,30.0,1.0,expected,262602.344292,32.469527,4.044919e+06,123.441128
303,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,1,...,122.678925,4.019943e+06,3007.903179,10.0,1.0,naive,272137.093188,32.216036,4.019943e+06,122.678925
305,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,45,...,124.094620,4.066333e+06,7149.002452,30.0,1.0,rollback,301925.210223,32.466066,4.066332e+06,124.094620
307,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,45,...,124.032479,4.064296e+06,7129.516815,100.0,1.0,expected,302162.494582,32.471554,4.064296e+06,124.032479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108968,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,18,...,5.482509,1.796509e+05,4.899477,10.0,1.0,expected,514234.752037,42.188129,2.421064e+05,5.482509
108969,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,23,...,3.439714,1.127125e+05,1.327019,100.0,1.0,expected,480536.545977,41.468061,1.963595e+05,3.439714
108972,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,1,...,5.744125,1.882235e+05,6.163179,100.0,1.0,rollback,616421.388552,45.339285,2.449270e+05,5.744125
108973,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,28.659882,9.391270e+05,116.954707,3.0,1.0,naive,335953.828373,34.752739,9.391270e+05,28.659882


In [12]:
df["num_niches"].unique()


array([4, 8, 1])

In [13]:
df["num_islands"].unique()


array([1024,    1])

In [14]:
df["tournament_size"].unique()


array([2, 1, 4])

In [15]:
df["a"].unique()


array(['reconstructed-tree', 'collapsed-phylogeny'], dtype=object)

In [16]:
df["regime"] = df.apply(
    lambda row: {
        (1, 1, 2): "plain",
        (1, 1, 1): "weak selection",
        (1, 1, 4): "strong selection",
        (1, 4, 2): "ecology",
        (1, 8, 2): "rich ecology",
        (1024, 1, 2): "spatial structure",
    }.get(
        tuple(row[["num_islands", "num_niches", "tournament_size"]]),
        np.nan,  # default
    ),
    axis="columns",
)
df = df.dropna(axis="index", subset=["regime"])
df["regime"].unique()


array(['rich ecology', 'spatial structure', 'ecology', 'weak selection',
       'strong selection', 'plain'], dtype=object)

In [17]:
df[df["a"] == "collapsed-phylogeny"]["regime"].unique()


array(['ecology', 'weak selection', 'plain', 'strong selection',
       'spatial structure', 'rich ecology'], dtype=object)

In [18]:
for setup, setup_df in df.groupby(sensitivity_analysis_variables):
    print(f"{setup=}")
    for regime, regime_df in setup_df.groupby(["regime"]):
        ground_truth = regime_df[(regime_df["a"] == "collapsed-phylogeny")][
            "average_origin_time_normalized"
        ].mean()
        print(f"   regime {ground_truth=}")

        for group, group_df in regime_df[
            (regime_df["a"] == "reconstructed-tree")
        ].groupby(["trie-postprocess", "resolution"]):
            err = (
                group_df["average_origin_time_normalized"].mean()
                - ground_truth
            )
            print("      ", group, len(group_df), err)


setup=(0, 'np.random.exponential')
   regime ground_truth=32765.118097476116
       ('expected', 3.0) 50 -0.025060253938136157
       ('expected', 10.0) 50 0.000995164049527375
       ('expected', 30.0) 50 -0.00018116798673872836
       ('expected', 100.0) 50 -0.0008352655859198421
       ('naive', 3.0) 50 -0.01729209265249665
       ('naive', 10.0) 50 0.007817926580173662
       ('naive', 30.0) 50 0.006332590721285669
       ('naive', 100.0) 50 0.005748161260271445
       ('rollback', 3.0) 50 -0.021631519517541165
       ('rollback', 10.0) 50 0.003895992922480218
       ('rollback', 30.0) 50 0.0024262215119961184
       ('rollback', 100.0) 50 0.0018417920473439153
   regime ground_truth=32766.119956053346
       ('expected', 3.0) 50 -0.009138803765381454
       ('expected', 10.0) 50 -0.0010293127015756909
       ('expected', 30.0) 50 -0.0007953567182994448
       ('expected', 100.0) 50 -0.0007405960095638875
       ('naive', 3.0) 50 -0.0020862453056906816
       ('naive', 10.0) 50 0.0